In [28]:
import pandas as pd
import numpy as np

Creating an array of the column names 

In [29]:
names = ['id', 'id_str','screen_name', 'location', 'description', 'url', 'followers_count', 'friends_count', 'listedcount','created_at','favourites_count','verified','statuses_count','lang','status','default_profile','default_profile_image','has_extended_profile','name','bot']

Reading the bots and non bots data from the CSV files and combining them into one dataframe

In [30]:
df = pd.read_csv('training_data_2_csv_UTF.csv',header=None,names=names,na_values='?',skiprows=1)
df_test = pd.read_csv('test_data_4_students.csv',header=None,names=names,na_values='?',skiprows=1)

df_test = df_test[0:575]

The codes that follow are used to extract certain features from the raw data obtained from the csv files.

The first feature that we are computing is the screen name length from the screen name column in the data

In [31]:
screen = list(df['screen_name'])
screen_length = []
for i in screen:
    screen_length.append(len(i))
df['screen_length'] = screen_length

# This is done for the test data as well
screen2 = list(df_test['screen_name'])
screen_length2 = []
for i in screen2:
    screen_length2.append(len(i))
df_test['screen_length'] = screen_length2

The second feature we are computing is the profile name length from the name colum in the data 

In [32]:
names = list(df['name'])
name_length = []
for i in names:
    name_length.append(len(i))
df['name_length'] = name_length

# This is done for the test data as well
names2 = list(df_test['name'])
name_length2 = []
for i in names2:
    name_length2.append(len(i))
df_test['name_length'] = name_length2

The third feature we are computing is the days since the account was created. We do this by creating a datetime 
object from the dates present in the created at column. The data contains various formats of data thus we need to
handle each seperately.

In [33]:
created_at = list(df['created_at'])
dates = list(df['created_at'])
number_days =[]
from datetime import datetime

datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

for i in dates:
    if i[0] == '"':
        i = i[1:-1]
    if i[-1] == '"':
        i = i[:-1]
    now = datetime.now()
    a = str(i.split(" ")[0])
    s1 = ""
    if(a.find('/',0,len(a))==-1):
        d1 = datetime.strptime(i,'%a %b %d %H:%M:%S +0000 %Y')
        delta = now - d1
        number_days.append(delta.days)
    else:
        try:
            d2 = datetime.strptime(i,'%d/%m/%Y %H:%M')
            delta = now - d2
        except ValueError:
            d2 = datetime.strptime(i,'%m/%d/%Y %H:%M')
            delta = now - d2
        number_days.append(delta.days)
df['number_days'] = number_days

# This is done for the test data as well
created_at = list(df_test['created_at'])
dates = list(df_test['created_at'])
number_days =[]
from datetime import datetime

datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

for i in dates:
    if i[0] == '"':
        i = i[1:-1]
    if i[-1] == '"':
        i = i[:-1]
    now = datetime.now()
    a = str(i.split(" ")[0])
    s1 = ""
    if(a.find('/',0,len(a))==-1):
        d1 = datetime.strptime(i,'%a %b %d %H:%M:%S +0000 %Y')
        delta = now - d1
        number_days.append(delta.days)
    else:
        try:
            d2 = datetime.strptime(i,'%d/%m/%Y %H:%M')
            delta = now - d2
        except ValueError:
            d2 = datetime.strptime(i,'%m/%d/%Y %H:%M')
            delta = now - d2
        number_days.append(delta.days)
df_test['number_days'] = number_days

The fourth feature we are computing is whether the account description contains the term "bot" in it. We store a value 
1 if the description contains the term bot and 0 otherwise

In [40]:
desc = df['description']
flag = []
for i in desc:
    i = str(i)
    #print i
    if i.find("bot")==-1:
        #print 0
        flag.append(0)
    else:
        #print 1
        flag.append(1)
df['desc_contains_bot']=flag

# This is done for the test data as well

desc = df_test['description']
flag = []
for i in desc:
    i = str(i)
    #print i
    if i.find("bot")==-1:
        #print 0
        flag.append(0)
    else:
        #print 1
        flag.append(1)
df_test['desc_contains_bot']=flag

The final dataframe we create has all the features that we will use in the classification model. The schema of this dataframe is as follows 

In [39]:
final = df[['followers_count','friends_count','listedcount','favourites_count','verified','statuses_count','default_profile','default_profile_image','has_extended_profile','screen_length','name_length','number_days','desc_contains_bot','bot']]
final_test = df_test[['followers_count','friends_count','listedcount','favourites_count','verified','statuses_count','default_profile','default_profile_image','has_extended_profile','screen_length','name_length','number_days','desc_contains_bot','bot']]

final.head(5)

,followers_count,friends_count,listedcount,favourites_count,verified,statuses_count,default_profile,default_profile_image,has_extended_profile,screen_length,name_length,number_days,desc_contains_bot,bot
0,1291,0,10,0,False,78554,True,False,False,16,21,126,0,1
1,1,349,0,38,False,31,True,False,False,9,11,491,0,1
2,1086,0,14,0,False,713,True,False,False,15,18,534,1,1
3,33,0,8,0,False,676,True,True,False,9,18,802,0,1
4,11,745,0,146,False,185,False,False,False,12,12,857,0,1


We have cleaned the training and test data in this part of the code.

In [41]:
final= final.dropna()
y = final['bot']
X = final[['followers_count','friends_count','listedcount','favourites_count','verified','statuses_count','default_profile','default_profile_image','has_extended_profile','screen_length','name_length','number_days','desc_contains_bot']]
train_features = X
train_label = y
test_features = final_test[['followers_count','friends_count','listedcount','favourites_count','verified','statuses_count','default_profile','default_profile_image','has_extended_profile','screen_length','name_length','number_days','desc_contains_bot']]

test_features['followers_count'].replace('None', 0, inplace=True)
test_features['friends_count'].replace('None', 0, inplace=True)
test_features['listedcount'].replace('None', 0, inplace=True)
test_features['favourites_count'].replace('None', 0, inplace=True)
test_features['statuses_count'].replace('None', 0, inplace=True)
test_features['default_profile'].replace('None', False, inplace=True)
test_features['default_profile_image'].replace('None', False, inplace=True)
test_features['has_extended_profile'].replace('None', False, inplace=True)
test_features['verified'].replace('None', False, inplace=True)

test_features['default_profile'].replace('FALSE', False, inplace=True)
test_features['default_profile_image'].replace('FALSE', False, inplace=True)
test_features['has_extended_profile'].replace('FALSE', False, inplace=True)
test_features['verified'].replace('FALSE', False, inplace=True)

test_features['default_profile'].replace('TRUE', True, inplace=True)
test_features['default_profile_image'].replace('TRUE', True, inplace=True)
test_features['has_extended_profile'].replace('TRUE', True, inplace=True)
test_features['verified'].replace('TRUE', True, inplace=True)

train_features['default_profile'].replace('FALSE', False, inplace=True)
train_features['default_profile_image'].replace('FALSE', False, inplace=True)
train_features['has_extended_profile'].replace('FALSE', False, inplace=True)
train_features['verified'].replace('FALSE', False, inplace=True)

train_features['default_profile'].replace('TRUE', True, inplace=True)
train_features['default_profile_image'].replace('TRUE', True, inplace=True)
train_features['has_extended_profile'].replace('TRUE', True, inplace=True)
train_features['verified'].replace('TRUE', True, inplace=True)


In [43]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50,max_features= 'sqrt',max_depth=20)
clf.fit(train_features, train_label)
pred = clf.predict(test_features)


df_print = pd.DataFrame()
df_print['id'] = df_test['id']
df_print['bot'] = pred4
df_print['id'] = df_print['id'].astype(int)
df_print.to_csv('output_final1.csv',index = False)
print df_print.dtypes
print df_print.head(10)

id     int64
bot    int64
dtype: object
                   id  bot
0          2281292622    0
1          2344040251    1
2           765871267    0
3          4772373433    0
4          1324548560    1
5          2561341789    1
6           347810134    1
7           856303860    1
8  832875000000000000    1
9            88856792    0


Implementing the Voting Classifier with DecisionTree, RandomForest and AdaBoost

In [46]:
col_names2_dt = ['followers_count','friends_count','listedcount','favourites_count','verified','statuses_count','default_profile','default_profile_image','has_extended_profile']
col_names2_rf = ['followers_count','friends_count','listedcount','favourites_count','verified','statuses_count','default_profile','default_profile_image','has_extended_profile','screen_length','name_length','number_days','desc_contains_bot']
col_names2_ad = ['followers_count','friends_count','listedcount','favourites_count','verified','statuses_count','default_profile','default_profile_image','has_extended_profile','screen_length','name_length','number_days','desc_contains_bot']

train_features_dt = train_features[col_names2_dt]
test_features_dt = test_features[col_names2_dt]

from sklearn import tree
clf1 = tree.DecisionTreeClassifier(min_samples_split=10, criterion='entropy', max_depth= 6)
clf1.fit(train_features_dt, train_label)
pred1 = clf1.predict(test_features_dt)


train_features_rf = train_features[col_names2_rf]
test_features_rf = test_features[col_names2_rf]
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(n_estimators=50)
clf2.fit(train_features_rf, train_label)
pred2 = clf2.predict(test_features_rf)


train_features_ad = train_features[col_names2_ad]
test_features_ad = test_features[col_names2_ad]
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier(min_samples_split=10, criterion='entropy', max_depth= 6)
clf3 = AdaBoostClassifier(n_estimators=500, base_estimator = DTC)
clf3.fit(train_features_ad, train_label)
pred3 = clf3.predict(test_features_ad)

from sklearn.ensemble import VotingClassifier

eclf1 = VotingClassifier(estimators=[('rf', clf1), ('ad', clf2), ('dt', clf3)], voting='hard',weights=[1,1,1])
eclf1.fit(train_features, train_label)
pred4 = eclf1.predict(test_features)
    
df_print = pd.DataFrame()
df_print['id'] = df_test['id']
df_print['bot'] = pred4
df_print['id'] = df_print['id'].astype(int)
df_print.to_csv('output_final_2.csv',index = False)
print df_print.dtypes
print df_print.head(10)

id     int64
bot    int64
dtype: object
                   id  bot
0          2281292622    0
1          2344040251    0
2           765871267    0
3          4772373433    0
4          1324548560    1
5          2561341789    1
6           347810134    1
7           856303860    1
8  832875000000000000    1
9            88856792    0


Here we are trying to gauge which parameters of the model give us the best results in terms of accuracy. We used grid search method from sklearn to do this. By specifying the range of parameters of the model grid search creates all permutations of parameters for this model and returns us the set of parameters which give the best results

In [ ]:
param_grid = {'max_depth' : [None, 10,20,30,40,50],
              'n_estimators' :[10,50,100,200,300,400,500,600,700,800,900,1000],
              'max_features' : ['auto','sqrt','log2',0.4,0.8,0.2],
            }
from sklearn import grid_search
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()
clf = grid_search.GridSearchCV(forest, param_grid)
clf.fit(train_features, train_label)
print clf.best_params_

/Users/rudhikakawar/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/rudhikakawar/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [10, 20, 30, 40, 50, 100, 150, 200, 300, 400, 500]
             }
from sklearn import grid_search
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier(min_samples_split=10, criterion='entropy', max_depth= 6)

ABC = AdaBoostClassifier(base_estimator = DTC)
clf = grid_search.GridSearchCV(ABC, param_grid=param_grid, scoring = 'roc_auc')
clf.fit(train_features, train_label)
print clf.best_params_